In [1]:
# Import necessary libraries

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import xarray as xr
import cartopy.crs as ccrs

import random
import time

from pycontrails import Flight
from alive_progress import alive_bar

from src.sampling import calcTotalDistance, samplePoint, generateFlight

In [2]:
"""df = pd.read_csv('flight_data/flightlist_20190101_20190131.csv.gz')
df.drop('number', axis=1, inplace=True)
df.drop('registration', axis=1, inplace=True)
df.drop('icao24', axis=1, inplace=True)
df.to_pickle('flight_data/flightlist_20190101_20190131.pkl')"""

"df = pd.read_csv('flight_data/flightlist_20190101_20190131.csv.gz')\ndf.drop('number', axis=1, inplace=True)\ndf.drop('registration', axis=1, inplace=True)\ndf.drop('icao24', axis=1, inplace=True)\ndf.to_pickle('flight_data/flightlist_20190101_20190131.pkl')"

In [3]:
n_samples = 100
n_flights = 100

In [ ]:
df = pd.read_pickle('flight_data/flightlist_20190101_20190131.pkl')

In [35]:
samples = np.empty((n_samples,2),int)
flights = []

print("Converting to list of flight objects...")
with alive_bar(n_samples) as bar:
    for i in range(0, n_flights):
        flights.append(generateFlight(df.iloc[i]))
        bar()

total_distance = calcTotalDistance(flights)

print(f"\nTotal distance flown by dataset: {total_distance/1000:.2f} km")

print("\nTaking samples...")
with alive_bar(n_samples) as bar:
    for i in range(0, n_samples):
        samples[i] = samplePoint(flights, total_distance)
        bar()

sample_index = np.arange(0, n_samples, 1)

longitudes   = np.empty(n_samples)
latitudes    = np.empty(n_samples)
altitudes    = np.empty(n_samples)
times        = np.empty(n_samples, dtype = 'datetime64[s]')
aircrafts    = np.empty(n_samples, dtype = 'str')

print("\nDetermining sample characteristics...")
with alive_bar(n_samples) as bar:
    for i in range(0, n_samples):
 
        longitudes[i]   = flights[samples[i][0]]['longitude'][samples[i][1]]
        latitudes[i]    = flights[samples[i][0]]['latitude'][samples[i][1]]
        altitudes[i]    = flights[samples[i][0]]['altitude'][samples[i][1]]
        times[i]        = flights[samples[i][0]]['time'][samples[i][1]]

        aircrafts[i]    = flights[samples[i][0]].attrs['aircraft_type']

        bar()

Converting to list of flight objects...
|████████████████████████████████████████| 100/100 [100%] in 0.2s (611.80/s) 

Total distance flown by dataset: 840606.01 km

Taking samples...
|████████████████████████████████████████| 100/100 [100%] in 0.9s (113.21/s) 

Determining sample characteristics...
|████████████████████████████████████████| 100/100 [100%] in 0.0s (223705.64/s) 
